In [1]:
import json, umap, utils, copy
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.spatial.distance import pdist, squareform
from python_tsp.exact import solve_tsp_dynamic_programming
from castle.algorithms import PC
import networkx as nx
from pgmpy.estimators import BicScore, MmhcEstimator
from itertools import product
from statistics import median

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chunyuko/.cache/huggingface/token
Login successful


2023-12-27 23:20:04,210 - /Users/chunyuko/Documents/anaconda3/envs/py_3_11/lib/python3.11/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2023-12-27 23:20:04,255 - /Users/chunyuko/Documents/anaconda3/envs/py_3_11/lib/python3.11/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [2]:
data = pd.read_csv("data/wine_dataset_variable_text.csv")
with open("data/wine_dataset_variable_text_embeddings.json", "r") as f:
    data["embeddings"] = [i[0] for i in json.load(f)]

In [3]:
umap_reducer = umap.UMAP(random_state=42)
umap_embedding = umap_reducer.fit_transform(data["embeddings"].tolist())

/Users/chunyuko/Documents/anaconda3/envs/py_3_11/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
data = pd.concat([data,pd.DataFrame(umap_embedding, columns=["emb_0", "emb_1"])], axis=1)

In [5]:
data["text"] = 4*data.query("variable == 'variable_name'")["value"].tolist()

In [6]:
px.scatter(data, x="emb_0", y="emb_1", color="variable", text = "text", width = 600, height = 400).\
    update_traces(textposition='top center').\
    update_layout(
            xaxis=dict(showgrid=False, title="", zeroline = False, showticklabels=False),
            yaxis=dict(showgrid=False, title="", zeroline = False, showticklabels=False),
            template="plotly_white")

In [7]:
def get_graphs(data, order):
    
    exp_data = data.copy()
    order = exp_data.columns[order]
    
    exp_data = exp_data[order]
    exp_array = np.array(exp_data)
    
    pc = PC()
    pc.learn(exp_data)
    graph = nx.DiGraph(np.array(pc.causal_matrix))
    
    if nx.is_directed_acyclic_graph(graph) == False:
        
        undirected_edges = {frozenset(i) for i in graph.edges() if graph.has_edge(*i[::-1])}
        undirected_edges = [([*i][0], [*i][1]) for i in [*undirected_edges]]
        directions = list(product([0, 1], repeat = len(undirected_edges)))

        graphs = []
        
        for i in directions:
            
            new_graph = copy.deepcopy(graph)
            
            for j, k in enumerate(i):
                
                l = undirected_edges[j][0]
                m = undirected_edges[j][1]
                
                new_graph.remove_edge(l, m)
                new_graph.remove_edge(m, l)
                
                if k > 0:
                    new_graph.add_edge(l, m)
                else:
                    new_graph.add_edge(m, l)
            
            if nx.is_directed_acyclic_graph(new_graph):
                graphs.append(new_graph)
    
    else:
        graphs = [graph]
    
    bic_calculator = BicScore(pd.DataFrame(exp_array))
    bics = [bic_calculator.score(i) for i in graphs]
    
    return graphs, bics

In [8]:
wine = utils.get_data_from_ucirepo(109).drop(["class_2", "class_3"], axis = 1)

In [9]:
random_orders = []

for i in range(100):
    random_orders.append(np.random.permutation([*range(len(wine.columns))]).tolist())

random_result = []

for i in random_orders:
    graph, bics = get_graphs(wine, i)
    if len(graph) > 0:
        random_result.append({"graphs": graph, "bics": bics})
        
print("{:,.0f}".format(median([min(i["bics"]) for i in random_result])))

-32,155,814,029


In [10]:
embedding_names = [*set(data.variable.tolist())]
embedding_orders = []

for i in embedding_names:
    embeddings = data[data.variable == i].embeddings.tolist()
    distance_matrix = squareform(pdist(embeddings))
    distance_matrix[:, 0] = 0
    order = solve_tsp_dynamic_programming(distance_matrix)[0]
    # order = order[::2] + order[1::2]
    embedding_orders.append(order)

embedding_results = []

for i in embedding_orders:
    graph, bics = get_graphs(wine, i)
    if len(graph) > 0:
        print("{:,.0f}".format(min(bics)))
    else :
        print("No DAG")

No DAG
-1,165,612,442
No DAG
-32,299,204,190


['column_name', 'name_description', 'variable_name', 'prompt']